In [ ]:
"""
Program Documentation: MMA Event Masterlist Data Cleaning Pipeline

This program is designed to clean and preprocess an MMA fight dataset, specifically an event masterlist. 
It performs several cleaning steps to ensure that key columns in the dataset are standardized, any missing values 
in critical fields are handled appropriately, and the final dataset is prepared for analysis. The pipeline focuses 
on cleaning columns such as 'weightclass', 'winner', 'height', and 'reach', as well as dropping rows with missing 
values in essential columns. The cleaned dataset is then saved to a new file.

Functionality Overview:

1. **Weightclass Cleaning**:
   - The function `clean_weightclass` is responsible for standardizing the values in the 'weightclass' column. 
     It maps various keywords in the 'weightclass' field (e.g., 'straw', 'fly', 'bantam', etc.) to a set of predefined, 
     valid weight classes (e.g., 'Strawweight', 'Flyweight', 'Bantamweight', etc.). If no valid match is found, the 
     value is set to `NaN`.

2. **Winner Column Cleaning (No Contest and Draw Handling)**:
   - The function `clean_no_contest_and_draws` processes the 'winner' column. It handles cases where the 'winner' value 
     is 'N/A', indicating a No Contest or Draw scenario. If the method of victory includes 'Decision', the 'winner' is 
     changed to 'Draw'. Otherwise, the value is updated to 'NC' (No Contest). This ensures that the 'winner' column 
     reflects accurate outcomes for fights that did not have a clear winner.

3. **Height and Reach Cleaning**:
   - The function `clean_height_and_reach` removes double quotes from the 'reach' columns (e.g., 'X"') and converts both 
     height and reach values to floats. This ensures that the data for fighters' height and reach are in a standardized 
     format for easier analysis.

4. **Dropping Rows with Missing Values**:
   - The function `drop_nans` removes rows with missing values in critical columns. It focuses on essential fields such 
     as 'event_name', 'event_date', 'winner', 'fighter_a_name', 'fighter_b_name', 'weightclass', 'method_of_victory', 
     'round_of_victory', 'time_of_victory', 'time_format', and 'gender'. The program prints the number of rows dropped 
     and identifies which columns caused the rows to be removed.

5. **Diagnostics and Unique Value Checks**:
   - The program prints unique values from both the 'weightclass' and 'method_of_victory' columns for diagnostic purposes, 
     allowing the user to verify that the cleaning steps were applied correctly. If no rows with NaN values are found in 
     'method_of_victory', the program will print a message indicating this.

6. **Saving the Cleaned Dataset**:
   - After all the cleaning steps are applied, the program saves the cleaned dataset to a specified file path. 
     The cleaned file is saved in CSV format, and the path to the saved file is printed at the end of the process.

Pipeline Execution:
- The file path to the dataset is specified in the 'file_path' variable, and the main pipeline function `main_pipeline` 
  is responsible for calling each of the cleaning steps in the correct order. The cleaned data is saved to a new file 
  after the entire cleaning process is complete.

Example Usage:
1. Specify the path to the dataset in the `file_path` variable.
2. Run the `main_pipeline(file_path)` function to clean and preprocess the data.
3. Check the console output to see progress and diagnostic messages, including any rows dropped due to missing values 
   and which columns caused the drops.
4. The cleaned data will be saved to a new file, and the path will be printed to the console.

Key Notes:
- The program assumes that the dataset has specific columns such as 'weightclass', 'method_of_victory', 'winner', 
  'fighter_a_height', 'fighter_b_height', etc.
- The handling of 'N/A' values in the 'winner' column assumes that 'N/A' signifies a No Contest or Draw.
- Height and reach values are cleaned by removing any double quotes and converting the remaining values to floats for 
  numerical analysis.

"""

In [2]:
## Intial Masterlist Cleaning -- Functional (10.06.2024)
import pandas as pd
import numpy as np

# Define the list of valid weight classes
valid_weightclasses = [
    'Strawweight', 'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'Light heavyweight', 'Heavyweight', 'Catchweight'
]

# Function to clean the 'weightclass' column based on keywords
def clean_weightclass(df):
    print("Cleaning weightclass column...")
    weightclass_mapping = {
        "straw": "Strawweight",
        "fly": "Flyweight",
        "bantam": "Bantamweight",
        "feather": "Featherweight",
        "light heavy": "Light heavyweight", 
        "light": "Lightweight",
        "welter": "Welterweight",
        "middle": "Middleweight",
        "heavy": "Heavyweight",
        "catch": "Catchweight"
    }

    def map_weightclass(weightclass):
        for keyword, valid_weightclass in weightclass_mapping.items():
            if keyword in weightclass.lower():
                return valid_weightclass
        return np.nan

    # Update the 'weightclass' column directly
    df['weightclass'] = df['weightclass'].apply(map_weightclass)
    print("Weightclass cleaning complete.")
    return df

# Function to clean the 'winner' column by handling 'N/A' for No Contests and Draws
def clean_no_contest_and_draws(df):
    print("Cleaning winner column for No Contest and Draw scenarios...")
    def update_winner(row):
        if row['winner'] == 'N/A':
            if 'Decision' in row['method_of_victory']:
                return 'Draw'
            else:
                return 'NC'
        return row['winner']

    df['winner'] = df.apply(update_winner, axis=1)
    print("Winner column cleaned for No Contest and Draw scenarios.")
    return df

# Function to clean height and reach columns by removing quotes and converting to float
def clean_height_and_reach(df):
    print("Cleaning height and reach columns...")
    def clean_value(value):
        if isinstance(value, str) and '"' in value:
            value = value.replace('"', '')
        try:
            return float(value)
        except ValueError:
            return None

    df['fighter_a_height'] = df['fighter_a_height'].apply(clean_value)
    df['fighter_a_reach'] = df['fighter_a_reach'].apply(clean_value)
    df['fighter_b_height'] = df['fighter_b_height'].apply(clean_value)
    df['fighter_b_reach'] = df['fighter_b_reach'].apply(clean_value)
    print("Height and reach columns cleaned.")
    return df

# Function to clean title_fight column based on time_format
def clean_title_fight(df):
    print("Cleaning title_fight column...")
    
    def update_title_fight(row):
        if row['title_fight'] == True and '5' not in str(row['time_format']):
            return False
        return row['title_fight']
    
    df['title_fight'] = df.apply(update_title_fight, axis=1)
    print("Title fight cleaning complete.")
    return df

# Function to clean the 'time_format' column by extracting the first character and converting it to an integer
def clean_time_format(df):
    print("Cleaning time_format column...")
    
    def extract_rounds(time_format):
        if isinstance(time_format, str):
            # Extract the first character (number of rounds)
            try:
                return int(time_format[0])
            except ValueError:
                return np.nan  # In case the first character isn't a number
        return np.nan
    
    df['time_format'] = df['time_format'].apply(extract_rounds)
    
    # Check for any NaN values post-extraction
    nan_time_format = df[df['time_format'].isna()]
    if not nan_time_format.empty:
        print(f"Warning: {len(nan_time_format)} rows have NaN values in 'time_format' after cleaning.")
    else:
        print("No NaN values in 'time_format' after cleaning.")
    
    print("Time format cleaning complete.")
    return df

# Function to drop rows with NaN values in specified columns and indicate which columns caused rows to be dropped
def drop_nans(df):
    print("Dropping rows with NaN values in critical columns...")
    required_columns = [
        'event_name', 'event_date', 'winner', 'fighter_a_name', 'fighter_b_name', 
        'weightclass', 'method_of_victory', 'round_of_victory', 'time_of_victory', 
        'time_format', 'gender', 'title_fight'
    ]
    initial_rows = len(df)
    
    # Identify rows that have NaN values in the required columns
    rows_with_nans = df[df[required_columns].isna().any(axis=1)]
    columns_with_nans = rows_with_nans[required_columns].isna().sum()
    
    # Drop rows with NaN values
    df_cleaned = df.dropna(subset=required_columns)
    
    rows_dropped = initial_rows - len(df_cleaned)
    
    if rows_dropped > 0:
        print(f"Number of rows dropped: {rows_dropped}")
        print(f"Columns responsible for dropped rows and their NaN counts:\n{columns_with_nans}")
    else:
        print("No rows dropped.")
    
    return df_cleaned



def main_pipeline(file_path):
    print(f"Loading dataset from {file_path}...")
    df = pd.read_csv(file_path, keep_default_na=False, low_memory=False)
    print("Dataset loaded.")

    # Step 1: Clean weightclass column
    df = clean_weightclass(df)
    print("Checking unique values of cleaned 'weightclass' column:")
    print(df['weightclass'].unique())

    # Step 2: Clean method_of_victory (diagnostics only, no cleaning required)
    print("Checking unique values from the 'method_of_victory' column:")
    print(df['method_of_victory'].unique())

    nan_method_of_victory = df[df['method_of_victory'].isna()]
    if nan_method_of_victory.empty:
        print("\nNo rows with NaN values in 'method_of_victory'.")
    else:
        print(f"\nRows with NaN values in 'method_of_victory': {len(nan_method_of_victory)}")

    # Step 3: Clean winner column for No Contest and Draw scenarios
    df = clean_no_contest_and_draws(df)

    # Step 4: Clean height and reach columns for both fighters
    df = clean_height_and_reach(df)

    # Step 5: Clean title_fight column based on time_format
    df = clean_title_fight(df)

    # Step 6: Clean time_format column
    df = clean_time_format(df)

    # Step 7: Drop rows with NaN values in critical columns
    df_cleaned = drop_nans(df)

    # Save the cleaned DataFrame
    output_file_path = r"C:\Users\EditZ\UFC Research\New Pipeline\event_masterlist_initial_clean.csv"
    df_cleaned.to_csv(output_file_path, index=False)
    print(f"\nCleaned data saved to: {output_file_path}")

# Specify the file path and run the main pipeline
file_path = r"C:\Users\EditZ\UFC Research\New Pipeline\(Dont Touch) event_masterlist (Updated 10.4.24).csv"
main_pipeline(file_path)


Loading dataset from C:\Users\EditZ\UFC Research\New Pipeline\(Dont Touch) event_masterlist (Updated 10.4.24).csv...
Dataset loaded.
Cleaning weightclass column...
Weightclass cleaning complete.
Checking unique values of cleaned 'weightclass' column:
['Lightweight' 'Middleweight' 'Featherweight' 'Welterweight'
 'Light heavyweight' 'Bantamweight' 'Flyweight' 'Strawweight'
 'Heavyweight' 'Catchweight' nan]
Checking unique values from the 'method_of_victory' column:
["TKO - Doctor's Stoppage" 'Decision - Unanimous' 'Decision - Split'
 'KO/TKO' 'Submission' 'Could Not Continue' 'Decision - Majority' 'DQ'
 'Overturned']

No rows with NaN values in 'method_of_victory'.
Cleaning winner column for No Contest and Draw scenarios...
Winner column cleaned for No Contest and Draw scenarios.
Cleaning height and reach columns...
Height and reach columns cleaned.
Cleaning title_fight column...
Title fight cleaning complete.
Cleaning time_format column...
No NaN values in 'time_format' after cleaning.
